<a href="https://colab.research.google.com/github/ZLY1223/Vaccine_LLM_Prompt_Engineering/blob/main/Chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install bitsandbytes

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxx"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import set_seed

In [ ]:
pip install openai

In [ ]:
import openai

# M0: Demographics Only Model

## 1. GPT-4o-mini

In [ ]:
import pandas as pd
import re
import time
import openai

client = openai.OpenAI(api_key="sk-xxxxxxx")

file_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx"
df = pd.read_excel(file_path, index_col=None)

# Set seed range
seeds = list(range(1000, 1010))

for seed in seeds:
    print(f"\n==== Running for seed {seed} ====\n")

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt"]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are simulating a human persona's decision-making process regarding whether to receive a COVID-19 vaccine."
                },
                {
                    "role": "user",
                    "content": task_prompt
                }
            ],
            seed=seed,
            temperature=0.5,
            max_tokens=1024
        )

        assistant_response = response.choices[0].message.content
        print(f"Row {idx} Task Response (seed={seed}):\n{assistant_response}\n")

        # Extract likelihood
        match_vaccinated = re.search(r"Likelihood of Choosing Vaccinated:\s*(\d+)%", assistant_response)
        match_not_vaccinated = re.search(r"Likelihood of Choosing Not Vaccinated:\s*(\d+)%", assistant_response)

        likelihood_vaccinated = int(match_vaccinated.group(1)) if match_vaccinated else "Extraction Failed"
        likelihood_not_vaccinated = int(match_not_vaccinated.group(1)) if match_not_vaccinated else "Extraction Failed"

        if isinstance(likelihood_vaccinated, int) and isinstance(likelihood_not_vaccinated, int):
            option_mapping = {
                likelihood_vaccinated: "Vaccinated",
                likelihood_not_vaccinated: "Not Vaccinated"
            }
            max_prob = max(likelihood_vaccinated, likelihood_not_vaccinated)
            selected_option = option_mapping[max_prob]
        else:
            selected_option = "Extraction Failed"

        task_results.append({
            "id": id,
            "Task_Prompt": task_prompt,
            "Task_Response": assistant_response,
            "Vaccination Decision": selected_option,
            "Likelihood of Choosing Vaccinated (%)": f"{likelihood_vaccinated}%",
            "Likelihood of Choosing Not Vaccinated (%)": f"{likelihood_not_vaccinated}%"
        })
        time.sleep(0.2)

    # Save every seed results
    task_results_df = pd.DataFrame(task_results)
    task_output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M0/Task_Results_Two_Options_M0_4omini_seed{seed}.xlsx"
    task_results_df.to_excel(task_output_path, index=False)

    print(f"Task responses (seed={seed}) saved to {task_output_path}")


Streaming output truncated to the last 5000 lines.
Row 286 Task Response (seed=1009):
```
Brief Reasoning: I believe vaccination is important for protection against COVID-19, especially considering the ongoing risks associated with the virus and the potential for serious illness. Additionally, being vaccinated contributes to the overall safety of my community. 
Likelihood of Choosing Vaccinated: 85%
Likelihood of Choosing Not Vaccinated: 15%
```

Row 287 Task Response (seed=1009):
```
Brief Reasoning: I believe vaccination is important for protection against COVID-19, especially considering the ongoing risks associated with the virus and the potential for serious illness. Additionally, being vaccinated contributes to the overall safety of my community. 
Likelihood of Choosing Vaccinated: 80%
Likelihood of Choosing Not Vaccinated: 20%
```

Row 288 Task Response (seed=1009):
```
Brief Reasoning: I believe that receiving the COVID-19 vaccine is important for my protection and the safety o

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

filtered_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"
results_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/Task_Results_Two_Options_M0_4omini.xlsx"

filtered_df = pd.read_excel(filtered_path)
task_results_df = pd.read_excel(results_path)

filtered_df["id"] = filtered_df["id"].astype(int)
task_results_df["id"] = task_results_df["id"].astype(int)

merged_df = task_results_df.merge(
    filtered_df[["id", "Task3_Original_Response_Two_Options"]],
    on="id",
    how="inner"
)

merged_df["true_label"] = merged_df["Task3_Original_Response_Two_Options"]
merged_df["predicted_label"] = merged_df["Vaccination Decision"]

print("Label Set:", sorted(set(merged_df["true_label"]) | set(merged_df["predicted_label"])))

eps = 1e-10
merged_df["prob_vaccinated"] = merged_df["Likelihood of Choosing Vaccinated (%)"].str.replace("%", "").astype(float) / 100
merged_df["true_binary"] = merged_df["true_label"].apply(lambda x: 1 if x == "Vaccinated" else 0)

merged_df["cross_entropy"] = -(
    merged_df["true_binary"] * np.log(merged_df["prob_vaccinated"] + eps) +
    (1 - merged_df["true_binary"]) * np.log(1 - merged_df["prob_vaccinated"] + eps)
)

entropy_by_class = merged_df.groupby("true_label")["cross_entropy"].mean()

report = classification_report(
    merged_df["true_label"],
    merged_df["predicted_label"],
    labels=["Vaccinated", "Not Vaccinated"],
    target_names=["Vaccinated", "Not Vaccinated"],
    digits=4
)

print("\nClassification Report:")
print(report)

print("\nBinary Cross Entropy Loss by Class:")
for label, loss in entropy_by_class.items():
    print(f"{label}: {loss:.4f}")

Label Set: ['Not Vaccinated', 'Vaccinated']

Classification Report:
                precision    recall  f1-score   support

    Vaccinated     0.8060    1.0000    0.8926       806
Not Vaccinated     0.0000    0.0000    0.0000       194

      accuracy                         0.8060      1000
     macro avg     0.4030    0.5000    0.4463      1000
  weighted avg     0.6496    0.8060    0.7194      1000


Binary Cross Entropy Loss by Class:
Not Vaccinated: 1.6130
Vaccinated: 0.1991


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 2. GPT-4.1

In [ ]:
import pandas as pd
import re
import time
import openai

client = openai.OpenAI(api_key="sk-xxxxxxx")

file_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx"
df = pd.read_excel(file_path, index_col=None)

# Set seed range
seeds = list(range(1000, 1010))

for seed in seeds:
    print(f"\n==== Running for seed {seed} ====\n")

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt"]

        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {
                    "role": "system",
                    "content": "You are simulating a human persona's decision-making process regarding whether to receive a COVID-19 vaccine."
                },
                {
                    "role": "user",
                    "content": task_prompt
                }
            ],
            seed=seed,
            temperature=0.5,
            max_tokens=1024
        )

        assistant_response = response.choices[0].message.content
        print(f"Row {idx} Task Response (seed={seed}):\n{assistant_response}\n")

        # Extract likelihood
        match_vaccinated = re.search(r"Likelihood of Choosing Vaccinated:\s*(\d+)%", assistant_response)
        match_not_vaccinated = re.search(r"Likelihood of Choosing Not Vaccinated:\s*(\d+)%", assistant_response)

        likelihood_vaccinated = int(match_vaccinated.group(1)) if match_vaccinated else "Extraction Failed"
        likelihood_not_vaccinated = int(match_not_vaccinated.group(1)) if match_not_vaccinated else "Extraction Failed"

        if isinstance(likelihood_vaccinated, int) and isinstance(likelihood_not_vaccinated, int):
            option_mapping = {
                likelihood_vaccinated: "Vaccinated",
                likelihood_not_vaccinated: "Not Vaccinated"
            }
            max_prob = max(likelihood_vaccinated, likelihood_not_vaccinated)
            selected_option = option_mapping[max_prob]
        else:
            selected_option = "Extraction Failed"

        task_results.append({
            "id": id,
            "Task_Prompt": task_prompt,
            "Task_Response": assistant_response,
            "Vaccination Decision": selected_option,
            "Likelihood of Choosing Vaccinated (%)": f"{likelihood_vaccinated}%",
            "Likelihood of Choosing Not Vaccinated (%)": f"{likelihood_not_vaccinated}%"
        })
        time.sleep(0.2)

    # Save every seed results
    task_results_df = pd.DataFrame(task_results)
    task_output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M0/Task_Results_Two_Options_M0_4.1_seed{seed}.xlsx"
    task_results_df.to_excel(task_output_path, index=False)

    print(f"Task responses (seed={seed}) saved to {task_output_path}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

filtered_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"
results_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/Task_Results_Two_Options_M0_4.1.xlsx"

filtered_df = pd.read_excel(filtered_path)
task_results_df = pd.read_excel(results_path)

filtered_df["id"] = filtered_df["id"].astype(int)
task_results_df["id"] = task_results_df["id"].astype(int)

merged_df = task_results_df.merge(
    filtered_df[["id", "Task3_Original_Response_Two_Options"]],
    on="id",
    how="inner"
)

merged_df["true_label"] = merged_df["Task3_Original_Response_Two_Options"]
merged_df["predicted_label"] = merged_df["Vaccination Decision"]

print("Label Set:", sorted(set(merged_df["true_label"]) | set(merged_df["predicted_label"])))

eps = 1e-10
merged_df["prob_vaccinated"] = merged_df["Likelihood of Choosing Vaccinated (%)"].str.replace("%", "").astype(float) / 100
merged_df["true_binary"] = merged_df["true_label"].apply(lambda x: 1 if x == "Vaccinated" else 0)

merged_df["cross_entropy"] = -(
    merged_df["true_binary"] * np.log(merged_df["prob_vaccinated"] + eps) +
    (1 - merged_df["true_binary"]) * np.log(1 - merged_df["prob_vaccinated"] + eps)
)

entropy_by_class = merged_df.groupby("true_label")["cross_entropy"].mean()

report = classification_report(
    merged_df["true_label"],
    merged_df["predicted_label"],
    labels=["Vaccinated", "Not Vaccinated"],
    target_names=["Vaccinated", "Not Vaccinated"],
    digits=4
)

print("\nClassification Report:")
print(report)

print("\nBinary Cross Entropy Loss by Class:")
for label, loss in entropy_by_class.items():
    print(f"{label}: {loss:.4f}")

# M1: Survey Model (Demogaphics + Prior Beliefs)

## 1. GPT-4o-mini

In [ ]:
import pandas as pd
import re
import time
import openai

client = openai.OpenAI(api_key="sk-xxxxxxx")

file_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M1.xlsx"
df = pd.read_excel(file_path, index_col=None)

seeds = list(range(1000, 1010))

for seed in seeds:
    print(f"\n==== Running for seed {seed} ====\n")

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt"]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are simulating a human persona's decision-making process regarding whether to receive a COVID-19 vaccine."
                },
                {
                    "role": "user",
                    "content": task_prompt
                }
            ],
            seed=seed,
            temperature=0.5,
            max_tokens=1024
        )

        assistant_response = response.choices[0].message.content
        print(f"Row {idx} Task Response (seed={seed}):\n{assistant_response}\n")

        # Extract likelihood
        match_vaccinated = re.search(r"Likelihood of Choosing Vaccinated:\s*(\d+)%", assistant_response)
        match_not_vaccinated = re.search(r"Likelihood of Choosing Not Vaccinated:\s*(\d+)%", assistant_response)

        likelihood_vaccinated = int(match_vaccinated.group(1)) if match_vaccinated else "Extraction Failed"
        likelihood_not_vaccinated = int(match_not_vaccinated.group(1)) if match_not_vaccinated else "Extraction Failed"

        if isinstance(likelihood_vaccinated, int) and isinstance(likelihood_not_vaccinated, int):
            option_mapping = {
                likelihood_vaccinated: "Vaccinated",
                likelihood_not_vaccinated: "Not Vaccinated"
            }
            max_prob = max(likelihood_vaccinated, likelihood_not_vaccinated)
            selected_option = option_mapping[max_prob]
        else:
            selected_option = "Extraction Failed"

        task_results.append({
            "id": id,
            "Task_Prompt": task_prompt,
            "Task_Response": assistant_response,
            "Vaccination Decision": selected_option,
            "Likelihood of Choosing Vaccinated (%)": f"{likelihood_vaccinated}%",
            "Likelihood of Choosing Not Vaccinated (%)": f"{likelihood_not_vaccinated}%"
        })
        time.sleep(0.2)

    task_results_df = pd.DataFrame(task_results)
    task_output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M1/Task_Results_Two_Options_M1_4omini_seed{seed}.xlsx"
    task_results_df.to_excel(task_output_path, index=False)

    print(f"Task responses (seed={seed}) saved to {task_output_path}")

Finished. File saved to: /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/task_inputs_two_options_M1_v1.xlsx


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

seed = 1000
filtered_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"
results_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/Task_Results_Two_Options_M1_4omini_seed{seed}.xlsx"

filtered_df = pd.read_excel(filtered_path)
task_results_df = pd.read_excel(results_path)

filtered_df["id"] = filtered_df["id"].astype(int)
task_results_df["id"] = task_results_df["id"].astype(int)

merged_df = task_results_df.merge(
    filtered_df[["id", "Task3_Original_Response_Two_Options"]],
    on="id",
    how="inner"
)

merged_df["true_label"] = merged_df["Task3_Original_Response_Two_Options"]
merged_df["predicted_label"] = merged_df["Vaccination Decision"]

print("Label Set:", sorted(set(merged_df["true_label"]) | set(merged_df["predicted_label"])))

eps = 1e-10
merged_df["prob_vaccinated"] = merged_df["Likelihood of Choosing Vaccinated (%)"].str.replace("%", "").astype(float) / 100
merged_df["true_binary"] = merged_df["true_label"].apply(lambda x: 1 if x == "Vaccinated" else 0)

merged_df["cross_entropy"] = -(
    merged_df["true_binary"] * np.log(merged_df["prob_vaccinated"] + eps) +
    (1 - merged_df["true_binary"]) * np.log(1 - merged_df["prob_vaccinated"] + eps)
)

entropy_by_class = merged_df.groupby("true_label")["cross_entropy"].mean()

report = classification_report(
    merged_df["true_label"],
    merged_df["predicted_label"],
    labels=["Vaccinated", "Not Vaccinated"],
    target_names=["Vaccinated", "Not Vaccinated"],
    digits=4
)

print("\nClassification Report:")
print(report)

print("\nBinary Cross Entropy Loss by Class:")
for label, loss in entropy_by_class.items():
    print(f"{label}: {loss:.4f}")

## 2. GPT-4.1

In [ ]:
import pandas as pd
import re
import time
import openai

client = openai.OpenAI(api_key="sk-xxxxxxx")

file_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M1.xlsx"
df = pd.read_excel(file_path, index_col=None)

seeds = list(range(1000, 1010))

for seed in seeds:
    print(f"\n==== Running for seed {seed} ====\n")

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt"]

        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {
                    "role": "system",
                    "content": "You are simulating a human persona's decision-making process regarding whether to receive a COVID-19 vaccine."
                },
                {
                    "role": "user",
                    "content": task_prompt
                }
            ],
            seed=seed,
            temperature=0.5,
            max_tokens=1024
        )

        assistant_response = response.choices[0].message.content
        print(f"Row {idx} Task Response (seed={seed}):\n{assistant_response}\n")

        # Extract likelihood
        match_vaccinated = re.search(r"Likelihood of Choosing Vaccinated:\s*(\d+)%", assistant_response)
        match_not_vaccinated = re.search(r"Likelihood of Choosing Not Vaccinated:\s*(\d+)%", assistant_response)

        likelihood_vaccinated = int(match_vaccinated.group(1)) if match_vaccinated else "Extraction Failed"
        likelihood_not_vaccinated = int(match_not_vaccinated.group(1)) if match_not_vaccinated else "Extraction Failed"

        if isinstance(likelihood_vaccinated, int) and isinstance(likelihood_not_vaccinated, int):
            option_mapping = {
                likelihood_vaccinated: "Vaccinated",
                likelihood_not_vaccinated: "Not Vaccinated"
            }
            max_prob = max(likelihood_vaccinated, likelihood_not_vaccinated)
            selected_option = option_mapping[max_prob]
        else:
            selected_option = "Extraction Failed"

        task_results.append({
            "id": id,
            "Task_Prompt": task_prompt,
            "Task_Response": assistant_response,
            "Vaccination Decision": selected_option,
            "Likelihood of Choosing Vaccinated (%)": f"{likelihood_vaccinated}%",
            "Likelihood of Choosing Not Vaccinated (%)": f"{likelihood_not_vaccinated}%"
        })
        time.sleep(0.2)

    task_results_df = pd.DataFrame(task_results)
    task_output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M1/Task_Results_Two_Options_M1_4.1_seed{seed}.xlsx"
    task_results_df.to_excel(task_output_path, index=False)

    print(f"Task responses (seed={seed}) saved to {task_output_path}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

seed = 1000
filtered_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/test_500/Chatgpt/Three_Task_Original_Response.xlsx"
results_path = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/Task_Results_Two_Options_M1_4.1_seed{seed}.xlsx"

filtered_df = pd.read_excel(filtered_path)
task_results_df = pd.read_excel(results_path)

filtered_df["id"] = filtered_df["id"].astype(int)
task_results_df["id"] = task_results_df["id"].astype(int)

merged_df = task_results_df.merge(
    filtered_df[["id", "Task3_Original_Response_Two_Options"]],
    on="id",
    how="inner"
)

merged_df["true_label"] = merged_df["Task3_Original_Response_Two_Options"]
merged_df["predicted_label"] = merged_df["Vaccination Decision"]

print("Label Set:", sorted(set(merged_df["true_label"]) | set(merged_df["predicted_label"])))

eps = 1e-10
merged_df["prob_vaccinated"] = merged_df["Likelihood of Choosing Vaccinated (%)"].str.replace("%", "").astype(float) / 100
merged_df["true_binary"] = merged_df["true_label"].apply(lambda x: 1 if x == "Vaccinated" else 0)

merged_df["cross_entropy"] = -(
    merged_df["true_binary"] * np.log(merged_df["prob_vaccinated"] + eps) +
    (1 - merged_df["true_binary"]) * np.log(1 - merged_df["prob_vaccinated"] + eps)
)

entropy_by_class = merged_df.groupby("true_label")["cross_entropy"].mean()

report = classification_report(
    merged_df["true_label"],
    merged_df["predicted_label"],
    labels=["Vaccinated", "Not Vaccinated"],
    target_names=["Vaccinated", "Not Vaccinated"],
    digits=4
)

print("\nClassification Report:")
print(report)

print("\nBinary Cross Entropy Loss by Class:")
for label, loss in entropy_by_class.items():
    print(f"{label}: {loss:.4f}")

KeyboardInterrupt: 

# M2: Demographics + Media Diet

## 1. GPT-4o-mini

In [ ]:
# Use demo + beliefs to infer media diet
import pandas as pd
import random
import re

task_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx")
mapping_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/Media Diet/gpt4omini_demographics_beliefs_results_v4_four_diets.xlsx")
media_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/media_diet_with_extracted_words_edited.xlsx")
matrix_df = pd.read_excel(
    "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/Media_Diets.xlsx",
    sheet_name="Media Diets",
    index_col=0
)

mapping_df["id"] += 1
matrix_df = matrix_df.rename(columns={
    "Misinfo": "Low Credibility",
})

task_df = task_df.merge(mapping_df[["id", "media_diet_label"]], left_on="id", right_on="id", how="left")

diet_to_article_quota = {
    diet: matrix_df.loc[diet][matrix_df.loc[diet] > 0].to_dict()
    for diet in matrix_df.index
}

def build_rag_prompt(task_prompt, diet, _):
    quota = diet_to_article_quota.get(diet, {})
    selected_articles = []

    for cat, count in quota.items():
        available = media_df[media_df["Category"] == cat][["Headline", "First_100_Words"]].dropna()
        if len(available) == 0:
            print(f"[Warning] No articles available for category '{cat}' (diet: {diet})")
            continue
        samples = available.sample(n=min(count, len(available)), random_state=random.randint(0, 10000))
        selected_articles.extend(list(zip(samples["Headline"], samples["First_100_Words"])))

    random.shuffle(selected_articles)

    disease_block = f"""% Media Diet Information
Assume today is June 29, 2022. Your decision about receiving a COVID-19 vaccine is primarily shaped by your demographics and prior beliefs. However, the kind of information you are exposed to in the media may also influence your thinking.
Below are five real-world news articles that reflect the type of media you are likely to encounter.
As you read them, consider:
- Do any of these articles reinforce your existing beliefs?
- Do any contradict them or introduce new concerns or reassurances?
Unless strongly challenged by the articles, your decision should remain aligned with your prior beliefs. Only adjust your position if a particular article clearly shifts your perception.
If one article is especially influential, briefly mention it in your reasoning.
"""

    media_context = "The articles are listed below:\n"
    for i, (title, content) in enumerate(selected_articles[:5]):
        media_context += f"Article {i+1}: {title.strip()}\n{content.strip()}\n"

    insert_point = task_prompt.find("% Task Prompt")
    return task_prompt if insert_point == -1 else (
        task_prompt[:insert_point] + disease_block + media_context + task_prompt[insert_point:]
    )

def safe_build(row):
    diet = row["media_diet_label"]
    if diet not in diet_to_article_quota:
        print(f"[Warning] Diet not found in quota matrix: {diet}")
    return build_rag_prompt(row["Task_Prompt"], diet, None)

for i in range(10):
    random.seed(1000 + i)

    task_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx")
    task_df = task_df.merge(mapping_df[["id", "media_diet_label"]], on="id", how="left")

    task_df["Task_Prompt_Full"] = task_df.apply(safe_build, axis=1)

    task_df["Task_Prompt_Full"] = task_df["Task_Prompt_Full"].apply(
        lambda text: re.sub(
            r"Now, based on the information above,",
            "Now, based on your demographics and the content of the five articles above,",
            text
        )
    )

    output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4omini_seed{1000+i}.xlsx"
    task_df[["id", "Task_Prompt_Full", "media_diet_label"]].to_excel(output_path, index=False)
    print(f"[Saved] Seed {1000 + i} -> {output_path}")

[Saved] Seed 1000 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4.1_seed1000.xlsx
[Saved] Seed 1001 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4.1_seed1001.xlsx
[Saved] Seed 1002 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4.1_seed1002.xlsx
[Saved] Seed 1003 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4.1_seed1003.xlsx
[Saved] Seed 1004 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4.1_seed1004.xlsx
[Saved] Seed 1005 -> /content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/me

In [ ]:
sample_prompt = task_df[task_df["media_diet_label"] == "Left Echochamber"].iloc[0]["Task_Prompt_Full"]
print(sample_prompt)


[Task Information]: You are a 27-year-old male. You do not hold a university degree and your income is above the country median.
% Media Diet Information
Assume today is June 29, 2022. Your decision about receiving a COVID-19 vaccine is primarily shaped by your demographics and prior beliefs. However, the kind of information you are exposed to in the media may also influence your thinking.
Below are five real-world news articles that reflect the type of media you are likely to encounter.
As you read them, consider:
- Do any of these articles reinforce your existing beliefs?
- Do any contradict them or introduce new concerns or reassurances?
Unless strongly challenged by the articles, your decision should remain aligned with your prior beliefs. Only adjust your position if a particular article clearly shifts your perception.
If one article is especially influential, briefly mention it in your reasoning.
The articles are listed below:
Article 1: Incompetence, Apathy, and Malice: One Mill

In [ ]:
import pandas as pd
import re
import time
import openai

client = openai.OpenAI(api_key="sk-xxxxxxx")

input_base = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/"
output_base = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M2/Media_four_diets/"

# seeds to run
seeds_to_run = list(range(1000, 1010))

for seed in seeds_to_run:
    input_file = f"{input_base}task_inputs_two_options_four_diets_M2_4omini_seed{seed}.xlsx"
    output_file = f"{output_base}Task_Results_Two_Options_Four_Diets_M2_4omini_seed{seed}.xlsx"

    try:
        df = pd.read_excel(input_file)
    except Exception as e:
        print(f"Failed to read {input_file}: {e}")
        continue

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt_Full"]

        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": "You are simulating a human persona's decision-making process regarding whether to receive a COVID-19 vaccine."
                    },
                    {
                        "role": "user",
                        "content": task_prompt
                    }
                ],
                seed=seed,
                temperature=0.5,
                max_tokens=1024
            )

            assistant_response = response.choices[0].message.content
            print(f"Seed {seed} - Row {idx} Response:\n{assistant_response}\n")

            match_vaccinated = re.search(r"[Ll]ikelihood of Choosing Vaccinated[:：]?\s*\**(\d+)\s*%?", assistant_response)
            match_not_vaccinated = re.search(r"[Ll]ikelihood of Choosing Not Vaccinated[:：]?\s*\**(\d+)\s*%?", assistant_response)

            l_vac = int(match_vaccinated.group(1)) if match_vaccinated else -1
            l_not = int(match_not_vaccinated.group(1)) if match_not_vaccinated else -1

            if l_vac == -1 or l_not == -1:
                decision = "Extraction Failed"
            else:
                decision = "Vaccinated" if l_vac >= l_not else "Not Vaccinated"

            task_results.append({
                "id": id,
                "Task_Prompt": task_prompt,
                "Task_Response": assistant_response,
                "Vaccination Decision": decision,
                "Likelihood of Choosing Vaccinated (%)": f"{l_vac}%" if l_vac >= 0 else "NA",
                "Likelihood of Choosing Not Vaccinated (%)": f"{l_not}%" if l_not >= 0 else "NA"
            })

        except Exception as e:
            print(f"Error on seed {seed}, row {idx}: {e}")
            task_results.append({
                "id": id,
                "Task_Prompt": task_prompt,
                "Task_Response": "API Error",
                "Vaccination Decision": "API Error",
                "Likelihood of Choosing Vaccinated (%)": "NA",
                "Likelihood of Choosing Not Vaccinated (%)": "NA"
            })
            time.sleep(1.5)  # wait before retrying next

        time.sleep(0.5)  # limit token per minute usage

    df_results = pd.DataFrame(task_results)
    df_results.to_excel(output_file, index=False)
    print(f"Seed {seed} finished. Results saved to {output_file}\n")


Streaming output truncated to the last 5000 lines.

Seed 1009 - Row 170 Response:
```
Brief Reasoning: While some articles highlight ongoing risks from COVID-19 and mention the dangers of long Covid and high death tolls, others show that many events no longer require vaccination and that people are making individual choices. My prior beliefs and demographics make me cautious about the necessity and long-term effects of the vaccine, and the articles did not strongly challenge these concerns.

Likelihood of Choosing Vaccinated: 35%
Likelihood of Choosing Not Vaccinated: 65%
```

Seed 1009 - Row 171 Response:
Brief Reasoning: Based on my age and the FDA's recent approval of a second booster for people over 50 (Article 1), I feel reassured that vaccination is important for my protection, especially with concerns about waning immunity. None of the other articles strongly challenge this view.

Likelihood of Choosing Vaccinated: 75%
Likelihood of Choosing Not Vaccinated: 25%

Seed 1009 - Row 

## 2. GPT-4.1

In [ ]:
# Use demo + beliefs to infer media diet
import pandas as pd
import random
import re

task_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx")
mapping_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Results/Media Diet/gpt4.1_demographics_beliefs_results_v4_four_diets.xlsx")
media_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/media_diet_with_extracted_words_edited.xlsx")
matrix_df = pd.read_excel(
    "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/Media Diet Data/Media_Diets.xlsx",
    sheet_name="Media Diets",
    index_col=0
)

mapping_df["id"] += 1
matrix_df = matrix_df.rename(columns={
    "Misinfo": "Low Credibility",
})

task_df = task_df.merge(mapping_df[["id", "media_diet_label"]], left_on="id", right_on="id", how="left")

diet_to_article_quota = {
    diet: matrix_df.loc[diet][matrix_df.loc[diet] > 0].to_dict()
    for diet in matrix_df.index
}

def build_rag_prompt(task_prompt, diet, _):
    quota = diet_to_article_quota.get(diet, {})
    selected_articles = []

    for cat, count in quota.items():
        available = media_df[media_df["Category"] == cat][["Headline", "First_100_Words"]].dropna()
        if len(available) == 0:
            print(f"[Warning] No articles available for category '{cat}' (diet: {diet})")
            continue
        samples = available.sample(n=min(count, len(available)), random_state=random.randint(0, 10000))
        selected_articles.extend(list(zip(samples["Headline"], samples["First_100_Words"])))

    random.shuffle(selected_articles)

    disease_block = f"""% Media Diet Information
Assume today is June 29, 2022. Your decision about receiving a COVID-19 vaccine is primarily shaped by your demographics and prior beliefs. However, the kind of information you are exposed to in the media may also influence your thinking.
Below are five real-world news articles that reflect the type of media you are likely to encounter.
As you read them, consider:
- Do any of these articles reinforce your existing beliefs?
- Do any contradict them or introduce new concerns or reassurances?
Unless strongly challenged by the articles, your decision should remain aligned with your prior beliefs. Only adjust your position if a particular article clearly shifts your perception.
If one article is especially influential, briefly mention it in your reasoning.
"""

    media_context = "The articles are listed below:\n"
    for i, (title, content) in enumerate(selected_articles[:5]):
        media_context += f"Article {i+1}: {title.strip()}\n{content.strip()}\n"

    insert_point = task_prompt.find("% Task Prompt")
    return task_prompt if insert_point == -1 else (
        task_prompt[:insert_point] + disease_block + media_context + task_prompt[insert_point:]
    )

def safe_build(row):
    diet = row["media_diet_label"]
    if diet not in diet_to_article_quota:
        print(f"[Warning] Diet not found in quota matrix: {diet}")
    return build_rag_prompt(row["Task_Prompt"], diet, None)

for i in range(10):
    random.seed(1000 + i)

    task_df = pd.read_excel("/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/task_inputs_two_options_M0.xlsx")
    task_df = task_df.merge(mapping_df[["id", "media_diet_label"]], on="id", how="left")

    task_df["Task_Prompt_Full"] = task_df.apply(safe_build, axis=1)

    task_df["Task_Prompt_Full"] = task_df["Task_Prompt_Full"].apply(
        lambda text: re.sub(
            r"Now, based on the information above,",
            "Now, based on your demographics and the content of the five articles above,",
            text
        )
    )

    output_path = f"/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/task_inputs_two_options_four_diets_M2_4.1_seed{1000+i}.xlsx"
    task_df[["id", "Task_Prompt_Full", "media_diet_label"]].to_excel(output_path, index=False)
    print(f"[Saved] Seed {1000 + i} -> {output_path}")

In [ ]:
import pandas as pd
import re
import time
import openai

client = openai.OpenAI(api_key="sk-xxxxxxx")

input_base = "/content/drive/MyDrive/LLM Vaccination Attitudes/Input/sample_1000/media_diet_selection/Media_four_diets/"
output_base = "/content/drive/MyDrive/LLM Vaccination Attitudes/Results/sample_1000/Chatgpt/M2/Media_four_diets/"

# seeds to run
seeds_to_run = list(range(1000, 1010))

for seed in seeds_to_run:
    input_file = f"{input_base}task_inputs_two_options_four_diets_M2_4.1_seed{seed}.xlsx"
    output_file = f"{output_base}Task_Results_Two_Options_Four_Diets_M2_4.1_seed{seed}.xlsx"

    try:
        df = pd.read_excel(input_file)
    except Exception as e:
        print(f"Failed to read {input_file}: {e}")
        continue

    task_results = []

    for idx, row in df.iterrows():
        id = row["id"]
        task_prompt = row["Task_Prompt_Full"]

        try:
            response = client.chat.completions.create(
                model="gpt-4.1",
                # model="gemini-2.5-flash",
                messages=[
                    {
                        "role": "system",
                        "content": "You are simulating a human persona's decision-making process regarding whether to receive a COVID-19 vaccine."
                    },
                    {
                        "role": "user",
                        "content": task_prompt
                    }
                ],
                seed=seed,
                temperature=0.5,
                max_tokens=1024
            )

            assistant_response = response.choices[0].message.content
            print(f"Seed {seed} - Row {idx} Response:\n{assistant_response}\n")

            match_vaccinated = re.search(r"[Ll]ikelihood of Choosing Vaccinated[:：]?\s*\**(\d+)\s*%?", assistant_response)
            match_not_vaccinated = re.search(r"[Ll]ikelihood of Choosing Not Vaccinated[:：]?\s*\**(\d+)\s*%?", assistant_response)

            l_vac = int(match_vaccinated.group(1)) if match_vaccinated else -1
            l_not = int(match_not_vaccinated.group(1)) if match_not_vaccinated else -1

            if l_vac == -1 or l_not == -1:
                decision = "Extraction Failed"
            else:
                decision = "Vaccinated" if l_vac >= l_not else "Not Vaccinated"

            task_results.append({
                "id": id,
                "Task_Prompt": task_prompt,
                "Task_Response": assistant_response,
                "Vaccination Decision": decision,
                "Likelihood of Choosing Vaccinated (%)": f"{l_vac}%" if l_vac >= 0 else "NA",
                "Likelihood of Choosing Not Vaccinated (%)": f"{l_not}%" if l_not >= 0 else "NA"
            })

        except Exception as e:
            print(f"Error on seed {seed}, row {idx}: {e}")
            task_results.append({
                "id": id,
                "Task_Prompt": task_prompt,
                "Task_Response": "API Error",
                "Vaccination Decision": "API Error",
                "Likelihood of Choosing Vaccinated (%)": "NA",
                "Likelihood of Choosing Not Vaccinated (%)": "NA"
            })
            time.sleep(1.5)  # wait before retrying next

        time.sleep(0.5)  # limit token per minute usage

    df_results = pd.DataFrame(task_results)
    df_results.to_excel(output_file, index=False)
    print(f"Seed {seed} finished. Results saved to {output_file}\n")
